In [55]:
import os
import pandas as pd
import numpy as np
import json

# get all json files that store our interactions with chatgpt
# these are the final, optimized files
json_files_final_run = [
    os.path.join(dp, f) for dp, dn, fn in os.walk("../data/prompts_new_format") for f in fn
    if ("final" in f ) and f.endswith(".json") and (not f.endswith(" 2.json"))]

total_files = len(json_files_final_run)

# read all the data into a pandas dataframe
li = []
for filename in json_files_final_run:
    json_data = json.load(open(filename))
    for i, iteration in enumerate(json_data):
        iteration["iteration"] = i
        iteration["file"] = filename
        df = pd.DataFrame(columns=["iteration", "prompt", "code", "tfsec", "number_of_issues", "file"])
        df = df.append(iteration, ignore_index=True)
        li.append(df)
frame_all_runs = pd.concat(li, axis=0, ignore_index=True)

/var/folders/bq/l2q_cb9j6bj21nlxmzykz44r0000gn/T/ipykernel_10339/2009297517.py:22: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/bq/l2q_cb9j6bj21nlxmzykz44r0000gn/T/ipykernel_10339/2009297517.py:22: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/bq/l2q_cb9j6bj21nlxmzykz44r0000gn/T/ipykernel_10339/2009297517.py:22: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/bq/l2q_cb9j6bj21nlxmzykz44r0000gn/T/ipykernel_10339/2009297517.py:22: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/var/folders/bq/l2q_cb9j6bj21nlxmzykz44r0000gn/T/ipykernel_10339/2009297517.py:22: FutureWarning:

The frame.append method is de

In [56]:
print(len(frame_all_runs))
frame_all_runs["number_of_issues"].replace("", np.nan)
frame_all_runs["number_of_issues"].dropna(inplace=True)
frame_all_runs = frame_all_runs[frame_all_runs["number_of_issues"].astype(bool)]
frame_all_runs["number_of_issues"] = frame_all_runs["number_of_issues"].astype(int)
frame_all_runs = frame_all_runs[frame_all_runs["number_of_issues"] >= 0]
print(len(frame_all_runs))
frame_all_runs

213
164
iteration           object
prompt              object
code                object
tfsec               object
number_of_issues     int64
file                object
dtype: object


,iteration,prompt,code,tfsec,number_of_issues,file
0,0,Can you create terraform configuration code in...,"provider ""aws"" {\n region = ""us-west-2""\n}\n\...","{""results"": [{""rule_id"": ""AVD-AWS-0131"", ""long...",2,../data/prompts_new_format/prompt4/versuch14/f...
1,1,"provider ""aws"" {\n region = ""us-west-2""\n}\n\...","provider ""aws"" {\n region = ""us-west-2""\n}\n\...","{""results"": []}",0,../data/prompts_new_format/prompt4/versuch14/f...
2,0,Can you create terraform configuration code in...,"provider ""aws"" {\n region = ""us-west-2""\n}\n\...","{""results"": []}",0,../data/prompts_new_format/prompt4/versuch13/f...
3,0,Can you create terraform configuration code in...,terraform {\n required_providers {\n aws =...,"{""results"": [{""rule_id"": ""AVD-AWS-0131"", ""long...",2,../data/prompts_new_format/prompt4/abbruch1/pr...
5,0,Can you create terraform configuration code in...,"provider ""aws"" {\n region = ""us-west-2""\n}\n\...","{""results"": [{""rule_id"": ""AVD-AWS-0131"", ""long...",2,../data/prompts_new_format/prompt4/versuch12/f...
...,...,...,...,...,...,...
206,2,"provider ""aws"" {\n region = ""us-west-2""\n}\n\...","resource ""aws_instance"" ""example"" {\n ami ...","{""results"": [{""rule_id"": ""AVD-AWS-0131"", ""long...",4,../data/prompts_new_format/prompt1/versuch2/pr...
208,0,Can you create terraform configuration code in...,"provider ""aws"" {\n region = ""us-west-2""\n}\n\...","{""results"": [{""rule_id"": ""AVD-AWS-0131"", ""long...",2,../data/prompts_new_format/prompt1/versuch5/pr...
209,1,"provider ""aws"" {\n region = ""us-west-2""\n}\n\...","provider ""aws"" {\n region = ""us-west-2""\n}\n\...","{""results"": [{""rule_id"": ""AVD-AWS-0131"", ""long...",3,../data/prompts_new_format/prompt1/versuch5/pr...
210,2,"provider ""aws"" {\n region = ""us-west-2""\n}\n\...","provider ""aws"" {\n region = ""us-west-2""\n}\n\...","{""results"": [{""rule_id"": ""AVD-AWS-0131"", ""long...",1,../data/prompts_new_format/prompt1/versuch5/pr...


In [57]:
frame_all_runs["number_of_issues"].mean()

3.591463414634146

In [58]:
frame_all_runs["number_of_issues"].std()

4.666568246042515

In [60]:
import plotly.express as px
px.histogram(frame_all_runs[frame_all_runs["iteration"] == 0], x="number_of_issues", nbins=40)

In [129]:
from scipy.stats import ks_2samp, ttest_ind

In [62]:
before = frame_all_runs[frame_all_runs["iteration"] == 0]

In [121]:
files = frame_all_runs["file"].unique()
before = []
after = []
for file in files:
    slice = frame_all_runs[frame_all_runs["file"].str.contains(file)]
    slice.sort_values("iteration", ascending=True)
    sequence = slice["number_of_issues"].to_list()
    before.append(sequence[0])
    # we only consider it when
    # 1) optimization iterations took place
    # 2) no optimization is necessary because no issue
    if (len(sequence) > 1) or (sequence[0] == 0):
        after.append(sequence[-1])


# After Optimization
Mean
Standard Deviation
Histogram: Number of issues per script

In [136]:
print("mu", sum(after)/(len(after)))
mu = sum(after)/len(after)
sigma = sum([(x-mu)**2 for x in before])/len(before)
print("sigma", sigma**0.5)
px.histogram(after, nbins=40)

mu 1.9827586206896552
sigma 4.475637580045902


In [137]:
print("mu", sum(before)/(len(before)))
mu = sum(before)/len(before)
sigma = sum([(x-mu)**2 for x in before])/len(before)
print("sigms", sigma**0.5)
px.histogram(before, nbins=40)

mu 3.9875
sigms 4.001542671270668


In [124]:
# less: H_0 = F(X) <= G(X)  H_1: F(X) > G(X)
ks_2samp(before, after, alternative="less")

KstestResult(statistic=0.680603448275862, pvalue=1.027151302838902e-15, statistic_location=1, statistic_sign=-1)

In [130]:
ttest_ind(before, after, equal_var=False)

Ttest_indResult(statistic=2.4727090987321323, pvalue=0.015026686448307423)

In [125]:
frame_all_runs[frame_all_runs["number_of_issues"] == 30]

,iteration,prompt,code,tfsec,number_of_issues,file
43,1,"provider ""aws"" {\n region = ""us-west-2""\n}\n\...","provider ""aws"" {\n region = ""us-west-2""\n}\n\...","{""results"": [{""rule_id"": ""AVD-AWS-0086"", ""long...",30,../data/prompts_new_format/prompt3/versuch15/f...


In [126]:
import numpy as np

In [127]:
x = np.random.normal(2, 1, 10)
y = np.random.normal(0, 1, 10)

In [128]:
# less: H0: F(X) >= G(X) H1: F(X) < G(X)
ks_2samp(x,y,alternative="less")

KstestResult(statistic=0.9, pvalue=0.00010825088224469028, statistic_location=0.537464696978641, statistic_sign=-1)

In [ ]:
# p value is small ->
# null hypothesis is rejected ->
# F(X) < G(X) ->
# (x=G(X), y=F(X))